In [2]:
import torch
from PIL import Image
import torchvision 
torch.__version__
print(torch.cuda.current_device())


0


In [ ]:
cuda = torch.device('cuda')

'''
image_path = '/home/medathati/Work/SpectralSelfSupervision/Data/ILSVRC/Data/CLS-LOC/train/n03944341/n03944341_7353.JPEG'
img = Image.open(image_path).convert('RGB')
img1  = torchvision.transform(img)
'''


I  = torch.rand(5,3,5,5,device=cuda)
I_fft = torch.rfft(I, signal_ndim=2, onesided = False, normalized=False)
I_mag = ((I_fft[:,:,:,:,0]**2+I_fft[:,:,:,:,1]**2)**0.5)

I_mag_nth = I_mag**(1-0.1)
I_fft[:,:,:,:,0] = I_fft[:,:,:,:,0]/I_mag_nth
I_fft[:,:,:,:,1] = I_fft[:,:,:,:,1]/I_mag_nth
# batch x channels x height x width

I_fft[I_fft!=I_fft]=0
I_hat = torch.irfft(I_fft, signal_ndim=2, onesided = False, normalized=False)

print("Shape of input tensor: ",I.shape)
print("Shape of the FFT output: ",I_fft.shape)
print("Shape of the root filter output: ",I_hat.shape)

print(I)
print("---I_ht---")
print(I_hat)

In [ ]:
a = torch.tensor([0.0,1.5,2.0,3.0],device=cuda)
b = torch.tensor([0.0,.75,1.0,1.5],device=cuda)
c = b/a
c[c!=c]=0

print(a**2)
print(a**1)
print(b/a)
print(c)



In [ ]:
a.type()

In [ ]:
I  = torch.rand(3,3,device=cuda)
I1 = torch.min(I, dim=0,keepdim=True)
print(I)
print(I1)



In [ ]:
class MinMaxScalerVectorized(object):
    """MinMax Scaler

    Transforms each channel to the range [a, b].

    Parameters
    ----------
    feature_range : tuple
        Desired range of transformed data.
    """

    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __call__(self, tensor):
        """Fit features

        Parameters
        ----------
        stacked_features : tuple, list
            List of stacked features.

        Returns
        -------
        tensor 
            A tensor with scaled features using requested preprocessor.
        """

        tensor = torch.stack(tensor)

        # Feature range
        a, b = self.feature_range

        dist = tensor.max(dim=0, keepdim=True)[0] - tensor.min(dim=0, keepdim=True)[0]
        dist[dist == 0.0] = 1.0
        scale = 1.0 / dist
        tensor.mul_(scale).sub_(tensor.min(dim=0, keepdim=True)[0])
        tensor.mul_(b - a).add_(a)

        return tensor

scaler = MinMaxScalerVectorized(feature_range=(-1, 1))
I2 = scaler(I)
print(I2)

In [26]:
#a = torch.randn(3, 4, 16, 16, 16, 16)
a = torch.randn(3, 3,3)

def MinMaxNormalize(X):
    X_channel_flat = X.view(*(X.size()[:-2]),1,-1)
    X_channel_min,_ = torch.min(X_channel_flat,len(X.size())-1, keepdim=True, out=None)
    X_channel_max,_ = torch.max(X_channel_flat,len(X.size())-1, keepdim=True, out=None)
    X_channel_den = X_channel_max - X_channel_min
    X_channel_den[X_channel_den==0] = 1.0 # To avoid division by zero
    X_normalized_flat = (X_channel_flat - X_channel_min)/X_channel_den
    X_normalized = X_normalized_flat.view(X.size())
    return X_normalized

print(a)
print(MinMaxNormalize(a))


tensor([[[ 0.7264,  1.3371,  0.1607],
         [ 0.0646,  0.6287, -0.8154],
         [-0.3210, -0.1670,  0.7265]],

        [[ 0.4167, -0.7599, -0.4026],
         [ 1.5075,  1.9788,  0.6498],
         [-0.0632, -0.5583,  0.6603]],

        [[-0.5028,  0.7215, -2.0491],
         [-0.3033, -1.5699, -0.1638],
         [-1.5514, -1.5809,  0.4457]]])
tensor([[[0.7163, 1.0000, 0.4535],
         [0.4088, 0.6709, 0.0000],
         [0.2297, 0.3013, 0.7163]],

        [[0.4296, 0.0000, 0.1305],
         [0.8279, 1.0000, 0.5147],
         [0.2544, 0.0736, 0.5186]],

        [[0.5581, 1.0000, 0.0000],
         [0.6301, 0.1730, 0.6805],
         [0.1796, 0.1690, 0.9004]]])
